### Equal-Weight Index Fund

#### Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

#### Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

#### Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Now it's time to import these stocks to our Jupyter Notebook file.

We need the following information after **Parsing**:

* Weight in index of each stock
* Price of each stock

In [2]:
INDEXES = {'SP500': 'sp500', 
           'NASDAQ': 'nasdaq100', 
           'DOWJONES': 'dowjones'}

In [3]:
from bs4 import BeautifulSoup
from decimal import Decimal

def index_parse(index):
    
    url = f'https://www.slickcharts.com/{index}'

    request = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(request.text, "lxml")
    
    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df = pd.read_html(str(stats))[0]
    df['% Chg'] = df['% Chg'].str.strip('()-%')
    df['% Chg'] = pd.to_numeric(df['% Chg'])
    df['Chg'] = pd.to_numeric(df['Chg'])

    return df

In [4]:
parse_data = index_parse(INDEXES['SP500'])
parse_data.head()

,#,Company,Symbol,Weight,Price,Chg,% Chg
0,1,Apple Inc.,AAPL,6.786297,175.25,3.06,1.78
1,2,Microsoft Corporation,MSFT,5.966591,315.30,1.03,0.33
2,3,Amazon.com Inc.,AMZN,3.562196,3305.00,75.28,2.33
3,4,Tesla Inc,TSLA,2.176526,1059.30,1.18,0.11
4,5,Alphabet Inc. Class A,GOOGL,2.109612,2795.73,22.34,0.81


#### Define our deposit value and read Excel Portfolio file

In [30]:
depo = 1000

In [6]:
while True:
    try:
        depo += float(input('Enter the value of your portfolio in USD '))
        break
    except Exception:
        print('Thats not a number! \n Try again:')

Enter the value of your portfolio in USD 1000


In [28]:
# read the file with all assets in portfolio
portfolio = pd.read_excel('portfolio.xlsx')

del portfolio['Unnamed: 0']

In [29]:
portfolio.head()

,asset,amount,price,total_sum
0,AAPL,0.387218,175.25,67.860000
1,MSFT,0.176406,315.30,55.620794
2,AMZN,0.009447,3305.00,31.221614
3,TSLA,0.017372,1059.30,18.402129
4,GOOGL,0.006241,2795.73,17.447494


#### Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [9]:
class CalculateSupply:
    
    def __init__(self, user_portfolio, parse_data, depo):
        self.portfolio = user_portfolio
        self.parse_data = parse_data
        self.depo = depo
        
        self.total_deposit = sum(self.portfolio['total_sum'])
        self.total_assets_sum = self.portfolio.groupby('asset').agg({'total_sum': 'sum'})
        
    # parse elements from source
    def _parse_elem(self, elem):
        symbol = elem[1][2]
        index_weight = round(Decimal(elem[1][3] / 100), 5)
        current_price = elem[1][4]
        
        return symbol, index_weight, current_price
    
    # checking current allocation
    def _check_allocation(self, symbol):
        try:
            total_asset_sum = self.total_assets_sum.query(f"asset == '{symbol}'")['total_sum'][0]

        except Exception:
            total_asset_sum = 0

        asset_portfolio_weight = total_asset_sum / (self.total_deposit + self.depo)
        
        return total_asset_sum, asset_portfolio_weight
    
    # calculating recommendation
    def _calculating_result(self, index_weight, total_asset_sum, current_price): 
        required_size = (self.total_deposit + self.depo) * float(index_weight)
        required_sum = float(required_size) - total_asset_sum
        
        if required_sum:
            available_amount = required_sum / current_price
            
        return available_amount, required_sum
    
    def run(self):
        for elem in self.parse_data.iterrows():
            # define necessary variables
            symbol, index_weight, current_price = self._parse_data(elem)
            total_asset_sum, asset_portfolio_weight = self._check_allocation(symbol)
            available_amount, required_sum = self._calculating_result(index_weight, total_asset_sum, current_price)

            # adding recommendation
            self.portfolio = self.portfolio.append(pd.DataFrame([[symbol, available_amount, 
                                                        current_price, 
                                                        available_amount * current_price]], 
                                                      columns=self.portfolio.columns))


            self.depo -= float(required_sum)

        return self.portfolio
        
calculator = CalculateSupply(portfolio, parse_data, depo)

In [31]:
# run our script
portfolio = calculator.run()
portfolio.head()

,asset,amount,price,total_sum
0,AAPL,0.387218,175.25,67.860000
0,MSFT,0.176406,315.30,55.620794
0,AMZN,0.009447,3305.00,31.221614
0,TSLA,0.017372,1059.30,18.402129
0,GOOGL,0.006241,2795.73,17.447494


#### Saving Our DataFrame to Excel

Saving our Excel file is very easy:

In [33]:
portfolio.to_excel('portfolio.xlsx')